In [ ]:
#Import python packages. Some may need to be installed using the Python Package Manager.

import os
import datetime
import exifread
from PIL import Image
import wikipedia

In [ ]:
#This is the only variable that needs to be set. It is the path to the folder of images.

path = "C:\\1_projects\\138_fedgis2021\\images\\"

In [ ]:
#This function converts the coordinates contained in the exif to decimal degrees.

def _convert_to_degress(value):
    d = float(value.values[0].num) / float(value.values[0].den)
    m = float(value.values[1].num) / float(value.values[1].den)
    s = float(value.values[2].num) / float(value.values[2].den)

    return d + (m / 60.0) + (s / 3600.0)

In [ ]:
#This code reads the exif data, scrapes wikipedia, and plots each location in a geodatabase.

arcpy.env.workspace = path
current_time = datetime.datetime.now().strftime("%B_%d_%Y_%I_%M_%S%p")
arcpy.CreateFileGDB_management(path, current_time + '.gdb')
SR = arcpy.SpatialReference(4326)
new_point = arcpy.CreateFeatureclass_management(path + current_time + '.gdb', "Pictures", 'POINT', spatial_reference=SR)
fc = new_point[0]
arcpy.AddField_management(fc, "Name", "TEXT", "", "", 100, "Name")
arcpy.AddField_management(fc, "Image_Link", "TEXT", "", "", 100, "Image_Link")
arcpy.AddField_management(fc, "Wiki_Link", "TEXT", "", "", 200, "Wiki_link")
    

for filename in os.listdir(path):
    if filename.endswith(".JPG") or filename.endswith(".PNG") or filename.endswith(".jpg") or filename.endswith(".png"):
        im = Image.open(os.path.join(path, filename))
        tags = {}
        with open(os.path.join(path, filename), 'rb') as f:
            tags = exifread.process_file(f, details=False)
        if "GPS GPSLatitude" in tags.keys():
            lat = _convert_to_degress(tags["GPS GPSLatitude"])
            latRef = tags["GPS GPSLatitudeRef"]
            lngRef = tags["GPS GPSLongitudeRef"]
            if str(latRef) == 'S':
                lat = -lat
            lng = _convert_to_degress(tags["GPS GPSLongitude"])
            if str(lngRef) == 'W':
                lng = -lng
            name_search = wikipedia.geosearch(lat, lng, results=1, radius=10000)
            name = name_search[0]
            wiki = "https://en.wikipedia.org/wiki/" + name.replace(" ", "_")
            with arcpy.da.InsertCursor(fc, ['SHAPE@', 'Name', "Image_Link", "Wiki_link"]) as cursor:
                coordinates = arcpy.Point(lng,lat)
                cursor.insertRow((coordinates, str(name).replace(",",""), filename, wiki))
                
print("Export complete.")